# 多头注意力
:label:`sec_multihead-attention`

In practice, given the same set of queries, keys, and values we may want our model to combine knowledge from different behaviors of the same attention mechanism, such as capturing dependencies of various ranges (e.g., shorter-range vs. longer-range) within a sequence. Thus, it may be beneficial to allow our attention mechanism to jointly use different representation subspaces of queries, keys, and values.

实际上，对于相同的查询集、键集和值集，我们可能希望模型能够结合同一注意力机制的不同行为所蕴含的知识，例如在一个序列中捕捉不同范围（例如短范围与长范围）的依赖关系。因此，允许注意力机制联合使用查询、键和值的不同表示子空间可能是有益的。

To this end, instead of performing a single attention pooling, queries, keys, and values can be transformed with $h$ independently learned linear projections. Then these $h$ projected queries, keys, and values are fed into attention pooling in parallel. In the end, $h$ attention-pooling outputs are concatenated and transformed with another learned linear projection to produce the final output. This design is called multi-head attention, where each of the $h$ attention pooling outputs is a head (Vaswani et al., 2017). Using fully connected layers to perform learnable linear transformations, Fig. 11.5.1 describes multi-head attention.

为此，可以使用 $h$ 个独立学习的线性投影分别对查询、键和值进行转换，而不是执行单次注意力池化。然后将这 $h$ 个投影后的查询、键和值并行输入注意力池化。最后，将 $h$ 个注意力池化的输出进行拼接，并通过另一个学习的线性投影转换以生成最终输出。这种设计称为多头注意力，其中每个注意力池化的输出都是一个头（Vaswani 等人，2017）。图 11.5.1 使用全连接层执行可学习的线性变换来描述多头注意力。

![多头注意力：多个头连结然后线性变换](../images/multi-head-attention.png)



## 模型

在实现多头注意力之前，让我们用数学语言将这个模型形式化地描述出来。
给定查询 $\mathbf{q} \in \mathbb{R}^{d_q}$ 、
键 $\mathbf{k} \in \mathbb{R}^{d_k}$ 和
值 $\mathbf{v} \in \mathbb{R}^{d_v}$，
每个注意力头 $\mathbf{h}_i$（ $i = 1, \ldots, h$ ）的计算方法为：

$$\mathbf{h}_i = f(\mathbf W_i^{(q)}\mathbf q, \mathbf W_i^{(k)}\mathbf k,\mathbf W_i^{(v)}\mathbf v) \in \mathbb R^{p_v},$$

其中，可学习的参数包括
$\mathbf W_i^{(q)}\in\mathbb R^{p_q\times d_q}$、
$\mathbf W_i^{(k)}\in\mathbb R^{p_k\times d_k}$和
$\mathbf W_i^{(v)}\in\mathbb R^{p_v\times d_v}$，
以及代表 `注意力汇聚的函数(attention pooling)` $f$。
$f$ 可以是 `additive attention` 或者 `scaled dot product attention`
多头注意力的输出需要经过另一个线性转换，它对应着 $h$ 个头连结后的结果，因此其可学习参数是 $\mathbf W_o\in\mathbb R^{p_o\times h p_v}$ ：

$$\mathbf W_o \begin{bmatrix}\mathbf h_1\\\vdots\\\mathbf h_h\end{bmatrix} \in \mathbb{R}^{p_o}.$$

Based on this design, each head may attend to different parts of the input. More sophisticated functions than the simple weighted average can be expressed.

基于这种设计，每个头都可能会关注输入的不同部分，可以表示比简单加权平均值更复杂的函数。

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

## 实现

在实现过程中通常[**选择缩放点积注意力作为每一个注意力头**]。
为了避免计算代价和参数代价的大幅增长，
我们设定 $p_q = p_k = p_v = p_o / h$。
值得注意的是，如果将查询、键和值的线性变换的输出数量设置为
$p_q h = p_k h = p_v h = p_o$，
则可以并行计算 $h$ 个头。
在下面的实现中，$p_o$ 是通过参数 `num_hiddens` 指定的。


In [19]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, num_hiddens, num_heads, dropout, bias=False, **kwargs):
        super().__init__()
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_k = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_v = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_o = nn.LazyLinear(num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        queries = self.transpose_qkv(self.W_q(queries))
        keys = self.transpose_qkv(self.W_k(keys))
        values = self.transpose_qkv(self.W_v(values))

        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(valid_lens, repeats=self.num_heads, dim=0)

        # output的形状:(batch_size*num_heads，查询的个数， num_hiddens/num_heads)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = self.transpose_output(output)
        return self.W_o(output_concat)

为了能够 **使多个头并行计算**，上面的 `MultiHeadAttention` 类将使用下面定义的两个转置函数。具体来说，`transpose_output` 函数反转了 `transpose_qkv` 函数的操作。


In [20]:
#@save
@d2l.add_to_class(MultiHeadAttention)
def transpose_qkv(self, X):
    """为了多注意力头的并行计算而变换形状"""

    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], self.num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数, num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads, 查询或者“键－值”对的个数, num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])

#@save
@d2l.add_to_class(MultiHeadAttention)
def transpose_output(self, X):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, self.num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

下面使用键和值相同的小例子来[**测试**]我们编写的`MultiHeadAttention`类。多头注意力输出的形状是（`batch_size`，`num_queries`，`num_hiddens`）。

In [21]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): LazyLinear(in_features=0, out_features=100, bias=False)
  (W_k): LazyLinear(in_features=0, out_features=100, bias=False)
  (W_v): LazyLinear(in_features=0, out_features=100, bias=False)
  (W_o): LazyLinear(in_features=0, out_features=100, bias=False)
)

In [24]:
batch_size, num_queries = 2, 4
num_kv_pairs, valid_lens = 6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kv_pairs, num_hiddens))
print(f"X=queries:\n{X.shape}")

result = attention(X, Y, Y, valid_lens)
print(f"\nattention(queries, Y, Y, valid_lens):\n{result.shape}")

d2l.check_shape(attention(X, Y, Y, valid_lens),
                (batch_size, num_queries, num_hiddens))

X=queries:
torch.Size([2, 4, 100])

attention(queries, Y, Y, valid_lens):
torch.Size([2, 4, 100])


## 小结

* 多头注意力融合了来自于多个注意力汇聚的不同知识，这些知识的不同来源于相同的查询、键和值的不同的子空间表示。
* 基于适当的张量操作，可以实现多头注意力的并行计算。